In [1]:
from __future__ import division, print_function
from keras import backend as K
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Dropout, Lambda
from keras.layers.merge import Concatenate
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Model, load_model
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.layers.advanced_activations import PReLU

import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras.optimizers import Adam
from keras.preprocessing import image
import time
from tqdm import tqdm
import pickle
from time import sleep
import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


1.13.1


In [2]:
def get_holiday_triples(dat):
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!GET TRIPLES CALLED!!!!!!!!!!!!!!!!!!!!!!!!!!")
        image_groups = {}
        counter = 50000
        for index, row in dat.iterrows():
            img_name = row['new_filename']
            group_name = row['artist']
            if group_name in image_groups:
                image_groups[group_name].append(img_name)
            else:
                image_groups[group_name] = [img_name]
            counter = counter - 1
            if counter == 0:
                break
            
        num_sims = 0
        image_triples = []
        group_list = sorted(list(image_groups.keys()))
        for i, g in enumerate(group_list):
                if num_sims % 100 == 0:
                        print("Generated {:d} pos + {:d} neg = {:d} total image triples"
                                    .format(num_sims, num_sims, 2*num_sims))
                images_in_group = image_groups[g]
                sim_pairs_it = itertools.combinations(images_in_group, 2)
                # for each similar pair, generate a corresponding different pair
                for ref_image, sim_image in sim_pairs_it:
                        image_triples.append((ref_image, sim_image, 1))
                        num_sims += 1
                        while True:
                                j = np.random.randint(low=0, high=len(group_list), size=1)[0]
                                if j != i:
                                        break
                        dif_image_candidates = image_groups[group_list[j]]
                        k = np.random.randint(low=0, high=len(dif_image_candidates), size=1)[0]
                        dif_image = dif_image_candidates[k]
                        image_triples.append((ref_image, dif_image, 0))
        print("Generated {:d} pos + {:d} neg = {:d} total image triples"
                    .format(num_sims, num_sims, 2*num_sims))
        return image_triples

def train_test_split(triples, splits):
        assert sum(splits) == 1.0
        split_pts = np.cumsum(np.array([0.] + splits))
        indices = np.random.permutation(np.arange(len(triples)))
        shuffled_triples = [triples[i] for i in indices]
        data_splits = []
        for sid in range(len(splits)):
                start = int(split_pts[sid] * len(triples))
                end = int(split_pts[sid + 1] * len(triples))
                data_splits.append(shuffled_triples[start:end])
        return data_splits

def get_image_data(image_name, folder):
    
    img_dir = "/home/lr_navin/Painter_Siamese/dataset/" + str(folder) + "/" + str(folder) + "/" 
    img = image.load_img(img_dir + image_name, target_size=(img_rows, img_cols))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)[0].transpose()

    return x

def batch_to_vectors(batch, vec_size):
        # Data Prep from Image - Feed to CNN
        img_colours, img_rows, img_cols = input_dim
        
        X1 = np.zeros((len(batch), img_colours, img_rows, img_cols))
        X2 = np.zeros((len(batch), img_colours, img_rows, img_cols))

        Y = np.zeros((len(batch), 2))
       
        for tid in range(len(batch)):
            
            split_1 = batch[tid][0].split('/')
            split_2 = batch[tid][1].split('/')

            X1[tid] = get_image_data(split_1[1], split_1[0])#vec_dict[name_1].reshape(vec_size)
            X2[tid] = get_image_data(split_2[1], split_2[0])#vec_dict[name_2].reshape(vec_size)
            Y[tid] = [1, 0] if batch[tid][2] == 0 else [0, 1]
        
        return ([X1, X2], Y)
        
def data_generator(triples, vec_size, batch_size=32, stamp_str = "None"):
#         print(stamp_str)
#         print("Data Gen----")
        while True:
            # shuffle once per batch
            indices = np.random.permutation(np.arange(len(triples)))
            num_batches = len(triples) // batch_size
            for bid in range(num_batches):
                batch_indices = indices[bid * batch_size : (bid + 1) * batch_size]
                batch = [triples[i] for i in batch_indices]
                yield batch_to_vectors(batch, vec_size)

def evaluate_model(model_file, test_gen):
        model_name = os.path.basename(model_file)
        model = load_model(model_file, custom_objects={'contrastive_loss': contrastive_loss})
        print("=== Evaluating model: {:s} ===".format(model_name))
        ytrue, ypred = [], []
        num_test_steps = len(test_gen) // BATCH_SIZE
        for i in range(num_test_steps):
                (X1, X2), Y = next(test_gen)
                Y_ = model.predict([X1, X2])
                ytrue.extend(np.argmax(Y, axis=1).tolist())
                ypred.extend(np.argmax(Y_, axis=1).tolist())
        accuracy = accuracy_score(ytrue, ypred)
        print("\nAccuracy: {:.3f}".format(accuracy))
        print("\nConfusion Matrix")
        print(confusion_matrix(ytrue, ypred))
        print("\nClassification Report")
        print(classification_report(ytrue, ypred))
        return accuracy

def get_model_file(data_dir, vector_name, merge_mode, borf):
        return os.path.join(data_dir, "models", "{:s}-{:s}-{:s}.h5"
                                                .format(vector_name, merge_mode, borf))

In [3]:
data_drive_1 = '/home/lr_navin/Painter_Siamese/'

img_dir = '/home/lr_navin/Painter_Siamese/dataset/train/train/'

img_rows = 256
img_cols = 256

COMPLEX = 1
W_INIT = 'he_normal'
L2_REG = 0.003
VECTORIZERS = ["InceptionV3"]
MERGE_MODES = ["Concat", "Euclidean"]
input_dim = (3, img_rows, img_cols)
BATCH_SIZE = 25
NUM_EPOCHS = 10

PENULTIMATE_SIZE = 2048
SOFTMAX_SIZE = 1584


In [4]:
# Simple CNN Network
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_base_network(input_dim):

    # input image dimensions
    img_colours, img_rows, img_cols = input_dim
    
    # number of convolutional filters to use
    nb_filters = 16
    # size of pooling area for max pooling
    nb_pool = 2
    # convolution kernel size
    nb_conv = 3
    model = Sequential()

    model.add(Conv2D(nb_filters, (nb_conv, nb_conv),
                     padding='same',
                     input_shape=(img_colours, img_rows, img_cols),
                     kernel_initializer=W_INIT,
                     kernel_regularizer=l2(l=L2_REG)))
    
    model.add(Activation('relu'))
    
    model.add(Conv2D(nb_filters, (nb_conv, nb_conv),
                     padding='same',
                     kernel_initializer=W_INIT,
                     kernel_regularizer=l2(l=L2_REG)))    
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
    model.add(Dropout(rate=0.5)) #0.25 #too much dropout and loss -> nan

    model.add(Flatten())
    
    model.add(Dense(64, input_shape=(input_dim,), activation='relu'))
    #model.add(Dropout(0.05)) #too much dropout and loss -> nan
    model.add(Dense(32, activation='relu'))
#     model.add(Dense(2, activation='relu'))


    return model


In [4]:
# Complex Network Try

def siamese_cnn(imgs_dim, compile_):
    model = Sequential()

    model.add(_convolutional_layer(nb_filter=16, input_shape=imgs_dim))
    model.add(BatchNormalization(axis=-1, input_shape=imgs_dim))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=16))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))

    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=32))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))

    model.add(_convolutional_layer(nb_filter=64))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=64))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=64))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))

    model.add(_convolutional_layer(nb_filter=128))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=128))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=128))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))

    model.add(_convolutional_layer(nb_filter=256))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=256))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(_convolutional_layer(nb_filter=256))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    model.add(MaxPooling2D(data_format="channels_first", pool_size=(2, 2)))
    model.add(Dropout(rate=0.5))

    model.add(Flatten())
    model.add(_dense_layer(output_dim=PENULTIMATE_SIZE))
    model.add(BatchNormalization(axis=-1))
    model.add(PReLU(alpha_initializer=W_INIT))
    
    model.add(_dense_layer(output_dim=1024))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))
    
    model.add(_dense_layer(output_dim=1024))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))
    
    model.add(_dense_layer(output_dim=1024))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))

    model.add(_dense_layer(output_dim=512))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))

    model.add(_dense_layer(output_dim=512))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))

    model.add(_dense_layer(output_dim=128))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))
    
    model.add(_dense_layer(output_dim=128))
    model.add(Dropout(rate=0.5))
    model.add(Activation("relu"))
    
#     model.add(_dense_layer(output_dim=2))
#     model.add(Activation("softmax"))

    if compile_:
        model.add(Dropout(rate=0.5))
        model.add(_dense_layer(output_dim=SOFTMAX_SIZE))
        model.add(BatchNormalization(axis=-1))
        model.add(Activation(activation='softmax'))
        return compile_model(model)

    return model


def _convolutional_layer(nb_filter, input_shape=None):
    if input_shape:
        return _first_convolutional_layer(nb_filter, input_shape)
    else:
        return _intermediate_convolutional_layer(nb_filter)


def _first_convolutional_layer(nb_filter, input_shape):
    return Conv2D(
        nb_filter, (3, 3), input_shape=input_shape,
        padding='same', kernel_initializer=W_INIT, kernel_regularizer=l2(l=L2_REG))


def _intermediate_convolutional_layer(nb_filter):
    return Conv2D(
        nb_filter, (3, 3), padding='same',
        kernel_initializer=W_INIT, kernel_regularizer=l2(l=L2_REG))


def _dense_layer(output_dim):
    return Dense(units=output_dim, kernel_regularizer=l2(l=L2_REG), kernel_initializer=W_INIT)


def compile_model(model):
    adam = Adam(lr=0.001)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=adam,
        metrics=['accuracy'])
    return model


In [6]:
# Data Prep 
scores = np.zeros((len(VECTORIZERS), len(MERGE_MODES)))

dat = pd.read_csv('/home/lr_navin/Painter_Siamese/train_info_custom.csv')
print("Train Data Size - " + str(len(dat)))


image_sets = get_holiday_triples(dat)

train_triples, val_triples = train_test_split(image_sets, splits=[0.9, 0.1])
print(len(train_triples), len(val_triples))


Train Data Size - 79433
!!!!!!!!!!!!!!!!!!!!!!!!!!GET TRIPLES CALLED!!!!!!!!!!!!!!!!!!!!!!!!!!
Generated 0 pos + 0 neg = 0 total image triples
Generated 127000 pos + 127000 neg = 254000 total image triples
Generated 539100 pos + 539100 neg = 1078200 total image triples
Generated 592800 pos + 592800 neg = 1185600 total image triples
Generated 862300 pos + 862300 neg = 1724600 total image triples
Generated 862300 pos + 862300 neg = 1724600 total image triples
Generated 1141000 pos + 1141000 neg = 2282000 total image triples
Generated 1484400 pos + 1484400 neg = 2968800 total image triples
Generated 2527300 pos + 2527300 neg = 5054600 total image triples
Generated 3540900 pos + 3540900 neg = 7081800 total image triples
Generated 3695207 pos + 3695207 neg = 7390414 total image triples
6651372 739042


In [7]:
#Generator Data - For NN

train_gen = data_generator(train_triples, input_dim, BATCH_SIZE)
val_gen = data_generator(val_triples, input_dim, BATCH_SIZE)


In [19]:
####################################
######### Simple CNN Model #########
####################################

# network definition
#base_network = create_base_network(input_dim)
print("Network Type ---- " + str(COMPLEX))
if COMPLEX == 1:
    base_network = siamese_cnn(input_dim, compile_=False)
else:
    base_network = create_base_network(input_dim)

input_a = Input(shape=input_dim)
input_b = Input(shape=input_dim)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(inputs=[input_a, input_b], outputs=distance)

#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam = Adam(lr=.001)
if COMPLEX == 1:
    model.compile(optimizer=adam, loss=contrastive_loss, metrics=["accuracy"])
else:
    model.compile(optimizer=adam, loss=contrastive_loss, metrics=["accuracy"])

model.summary()


print('concatnated model')
best_model_name = get_model_file(data_drive_1, "inceptionv3", "cat", "best")
checkpoint = ModelCheckpoint(best_model_name, save_best_only=True)
train_steps_per_epoch = len(train_triples) // BATCH_SIZE
val_steps_per_epoch = len(val_triples) // BATCH_SIZE
print("STEPS INFO - " + str(train_steps_per_epoch) + " AND " + str(val_steps_per_epoch)) 
history = model.fit_generator(train_gen,
                              steps_per_epoch=train_steps_per_epoch, 
                              epochs=NUM_EPOCHS, 
                              validation_data=val_gen,
                              validation_steps=val_steps_per_epoch,
                              callbacks=[checkpoint])

final_model_name = get_model_file(data_drive_1, "inceptionv3", "cat", "final")
model.save(final_model_name)

Network Type ---- 1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 3, 256, 256)  0                                            
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, 128)          13351872    input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 128)          0           sequential_7[1][0]      

KeyboardInterrupt: 

In [5]:
def data_generator(triples, vec_size, vec_dict, batch_size):
    while True:
        # shuffle once per batch
        indices = np.random.permutation(np.arange(len(triples)))
        num_batches = len(triples) // batch_size
        for bid in range(num_batches):
            batch_indices = indices[bid * batch_size: (bid + 1) * batch_size]
            batch = [triples[i] for i in batch_indices]
            yield batch_to_vectors(batch, vec_size, vec_dict)



# Best Model
best_model_name = get_model_file(data_drive_1, "inceptionv3", "cat", "best")
# Final Model
final_model_name = get_model_file(data_drive_1, "inceptionv3", "cat", "final")


test_data_seen = pd.read_csv('/home/lr_navin/Painter_Siamese/Evaluate_Baseline/info_data/test-train_test.csv')
test_data_unseen = pd.read_csv('/home/lr_navin/Painter_Siamese/Evaluate_Baseline/info_data/test_info-test_only.csv')
test_data_mix = pd.read_csv('/home/lr_navin/Painter_Siamese/Evaluate_Baseline/info_data/test_info.csv')

test_triples_seen = get_holiday_triples(test_data_seen)
test_triples_unseen = get_holiday_triples(test_data_unseen)
test_triples_mix = get_holiday_triples(test_data_mix)

# Testing Model's Performance - 3 types of test datasets
test_triples_seen_gen   = data_generator(test_triples_seen, input_dim, BATCH_SIZE)
test_triples_unseen_gen = data_generator(test_triples_unseen, input_dim, BATCH_SIZE)
test_triples_mix_gen    = data_generator(test_triples_mix, input_dim, BATCH_SIZE)


# Best Model
# Seen Classes Model - 
final_accuracy = evaluate_model(best_model_name, test_triples_seen_gen)

# UnSeen Classes Model - 
final_accuracy = evaluate_model(best_model_name, test_triples_unseen_gen)

# Mix Classes Model - 
final_accuracy = evaluate_model(best_model_name, test_triples_mix_gen)


# Final Model
# Seen Classes Model - 
final_accuracy = evaluate_model(final_model_name, test_triples_seen_gen)

# UnSeen Classes Model - 
final_accuracy = evaluate_model(final_model_name, test_triples_unseen_gen)

# Mix Classes Model - 
final_accuracy = evaluate_model(final_model_name, test_triples_mix_gen)

!!!!!!!!!!!!!!!!!!!!!!!!!!GET TRIPLES CALLED!!!!!!!!!!!!!!!!!!!!!!!!!!
Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 5500 pos + 5500 neg = 11000 total image triples
Generated 36900 pos + 36900 neg = 73800 total image triples
Generated 121600 pos + 121600 neg = 243200 total image triples
Generated 158700 pos + 158700 neg = 317400 total image triples
Generated 159600 pos + 159600 neg = 319200 total image triples
Generated 253900 pos + 253900 neg = 507800 total image triples
Generated 284800 pos + 284800 neg = 569600 total image triples
Generated 326200 pos + 326200 neg = 652400 total image triples
Generated 342600 pos + 342600 neg = 685200 total image triples
Generated 342600 pos + 342600 neg = 685200 total image triples
Generated 396000 pos + 396000 neg = 792000 total image triples
Generated 417700 pos + 417700 neg = 835400 total image triples
Generated 460100 pos + 460100 neg = 920200 total image triples
Generated 503347 pos +

TypeError: object of type 'generator' has no len()